In [54]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [55]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)


In [56]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 1. connect to SP

In [57]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


In [58]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection 0da802ed-ef09-4774-bea6-9987effe0009 in State invitation
Connection ID 0da802ed-ef09-4774-bea6-9987effe0009
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '7ed3d9da-83a3-440b-8c6e-64bbbc0c5da2', 'label': 'Alice', 'recipientKeys': ['2NXtAKWpEXaBsjveidkvu4rtjhMV3PWwCctbEj3Zicmu'], 'serviceEndpoint': 'http://192.168.65.3:8020'}
Connection Handler Called
Connection 0da802ed-ef09-4774-bea6-9987effe0009 in State request


### switch to SP 

In [59]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 0da802ed-ef09-4774-bea6-9987effe0009 in State response
ACCEPT REQUEST
{'routing_state': 'none', 'connection_id': '0da802ed-ef09-4774-bea6-9987effe0009', 'state': 'response', 'their_role': 'invitee', 'invitation_key': '2NXtAKWpEXaBsjveidkvu4rtjhMV3PWwCctbEj3Zicmu', 'my_did': 'P28K76EisDuQXgK1BBJc8M', 'their_label': 'Carol', 'rfc23_state': 'response-sent', 'their_did': '8KjR51tbaUQuYg7knMDfoZ', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 10:15:07.285538Z', 'updated_at': '2023-01-03 10:15:45.947474Z'}
state response


In [60]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '2d0a3c7c-2bc6-4596-96ca-feeb540de25f'}
Connection Handler Called
Connection 0da802ed-ef09-4774-bea6-9987effe0009 in State active


In [61]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
sp_connection = connection["connection_id"]
print(sp_connection)

{'routing_state': 'none', 'connection_id': '0da802ed-ef09-4774-bea6-9987effe0009', 'state': 'active', 'their_role': 'invitee', 'invitation_key': '2NXtAKWpEXaBsjveidkvu4rtjhMV3PWwCctbEj3Zicmu', 'my_did': 'P28K76EisDuQXgK1BBJc8M', 'their_label': 'Carol', 'rfc23_state': 'completed', 'their_did': '8KjR51tbaUQuYg7knMDfoZ', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 10:15:07.285538Z', 'updated_at': '2023-01-03 10:15:48.576060Z'}
Is Active? active
0da802ed-ef09-4774-bea6-9987effe0009


In [5]:
# sp_connection = 'e6557bae-b92b-4ad7-8c55-b91d496f6ce1'

In [64]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'routing_state': 'none', 'connection_id': '29da4a10-2751-494f-8d9c-bb93f4054974', 'state': 'active', 'their_role': 'invitee', 'invitation_key': '2djco6gS9tNs7Hw6cJ3wy4ddoejfYECPHQeiip56fBxt', 'my_did': 'AJKWXtEvaRXNkmXrTG9FxV', 'their_label': 'Carol', 'rfc23_state': 'completed', 'their_did': 'TUMFuVbpwwgMTnCH3fxty', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 09:34:00.646895Z', 'updated_at': '2023-01-03 09:34:33.786419Z'}, {'routing_state': 'none', 'connection_id': 'f7ea7bd3-1b80-4106-aeb4-5b826b8495a0', 'state': 'active', 'their_role': 'invitee', 'invitation_key': '7a5RP5ipXyM1Bra395Xn1ANni38p53F4Hg3nARjpYYw5', 'my_did': '2jtbrtAtQ5i6CuoZXNQTW1', 'their_label': 'Bob', 'rfc23_state': 'completed', 'their_did': 'B9LWxnPaVKusAdELrcZjJ2', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 09:35:20.240337Z', 'updated_at': '2023-01-03 09:38:35.537670Z'}, {'routing_state': 'none', 'connection_id': '0da802ed-ef09-4774-bea6-9987ef

In [63]:
response = await agent_controller.connections.remove_connection('031f3c7c-1b71-49b8-aded-1fd8bea8f3d8')

## 6. connect to MUSIC

In [70]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection 3c26b35c-565a-46cd-98eb-ac0337a8c8ce in State invitation
Connection ID 3c26b35c-565a-46cd-98eb-ac0337a8c8ce
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'ff2a54fa-b5c7-40b5-98f9-1c4fab300b2f', 'label': 'Alice', 'recipientKeys': ['EASvR9t7Dkb8pUta1dDcpZwDJj32zCLkXpjFvPotNTWx'], 'serviceEndpoint': 'http://192.168.65.3:8020'}
Connection Handler Called
Connection 3c26b35c-565a-46cd-98eb-ac0337a8c8ce in State request


### switch to MUSIC

In [71]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 3c26b35c-565a-46cd-98eb-ac0337a8c8ce in State response
ACCEPT REQUEST
{'routing_state': 'none', 'connection_id': '3c26b35c-565a-46cd-98eb-ac0337a8c8ce', 'state': 'response', 'their_role': 'invitee', 'invitation_key': 'EASvR9t7Dkb8pUta1dDcpZwDJj32zCLkXpjFvPotNTWx', 'my_did': 'US6mN2w2jdYCrwiBQdBzQE', 'their_label': 'Bob', 'rfc23_state': 'response-sent', 'their_did': 'KBtYiK8eaWNKEtnVWAR64h', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 11:31:01.560386Z', 'updated_at': '2023-01-03 11:31:55.769227Z'}
state response


In [72]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '654a5d95-9c3d-4df6-a969-11dc01ee125f'}
Connection Handler Called
Connection 3c26b35c-565a-46cd-98eb-ac0337a8c8ce in State active


In [73]:
connection = await agent_controller.connections.get_connection(music_connection)
print(connection)
print("Is Active?", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

{'routing_state': 'none', 'connection_id': 'f7ea7bd3-1b80-4106-aeb4-5b826b8495a0', 'state': 'active', 'their_role': 'invitee', 'invitation_key': '7a5RP5ipXyM1Bra395Xn1ANni38p53F4Hg3nARjpYYw5', 'my_did': '2jtbrtAtQ5i6CuoZXNQTW1', 'their_label': 'Bob', 'rfc23_state': 'completed', 'their_did': 'B9LWxnPaVKusAdELrcZjJ2', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 09:35:20.240337Z', 'updated_at': '2023-01-03 09:38:35.537670Z'}
Is Active? active
f7ea7bd3-1b80-4106-aeb4-5b826b8495a0


## 7. exchange DID with SP and MUSIC

In [74]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
sp_did = 'None'
music_did = 'None'

my DID PQRXDxdGqQGSZ8z69p4xZP


In [75]:
def messages_handler(payload):
    global sp_did
    global music_did
    connection_id = payload["connection_id"]
    did = payload["content"]
    if did[:3] == 'sp_':
        sp_did = did[4:]
        print('sp_did', sp_did)
    else:
        music_did = did[6:]
        print('music_did', music_did)
        
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, my_did))
    print("Handle message", payload, connection_id)
    
    
    

message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

Subscribing too: basicmessages


music_did gZj39Y8JGQ5GVQp2y6zFH
Handle message {'connection_id': '3c26b35c-565a-46cd-98eb-ac0337a8c8ce', 'message_id': 'e0a07e73-cafc-4a13-bee5-207daf0dcb31', 'content': 'music_gZj39Y8JGQ5GVQp2y6zFH', 'state': 'received'} 3c26b35c-565a-46cd-98eb-ac0337a8c8ce


In [52]:
# message = my_did
# response = await agent_controller.messaging.send_message(music_connection, message)
# print(response)

{}
music_did gZj39Y8JGQ5GVQp2y6zFH
Handle message {'connection_id': 'f7ea7bd3-1b80-4106-aeb4-5b826b8495a0', 'message_id': '40c0fc00-a2ed-4c63-be0b-9f2c3ad161a6', 'content': 'music_gZj39Y8JGQ5GVQp2y6zFH', 'state': 'received'} f7ea7bd3-1b80-4106-aeb4-5b826b8495a0
sp_did 8HWQruWXrv8s6BBLXm53B
Handle message {'connection_id': '29da4a10-2751-494f-8d9c-bb93f4054974', 'message_id': '13072c3a-ea6d-4382-bb38-069ea2ea1826', 'content': 'sp_X8HWQruWXrv8s6BBLXm53B', 'state': 'received'} 29da4a10-2751-494f-8d9c-bb93f4054974
music_did gZj39Y8JGQ5GVQp2y6zFH
Handle message {'connection_id': 'f7ea7bd3-1b80-4106-aeb4-5b826b8495a0', 'message_id': 'c89bf47d-ea34-4d1c-9c09-63fdab3aeea3', 'content': 'music_gZj39Y8JGQ5GVQp2y6zFH', 'state': 'received'} f7ea7bd3-1b80-4106-aeb4-5b826b8495a0


## issue new VC to MUSIC and SP

In [41]:
agent_controller.remove_all_listeners("basicmessages")

In [17]:
revoke = False

In [18]:
def request_handler(payload):
    global revoke
    connection_id = payload["connection_id"]
    if payload['content'] == 'True': 
        revoke = True
    send_task = asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(sp_connection, "revoke: "+str(revoke)))
    print("Handle request", payload, connection_id)


request_listener = {
    "handler": request_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([request_listener], defaults=True)

Subscribing too: basicmessages


### switch to SP

In [20]:
print(revoke)

True


### revoke old VC from MUSIC

In [37]:
if revoke:
    ## revoke the previous VC
    

### issue new VC to MUSIC

In [30]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_credential


In [31]:
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did=my_did)
print(created_schema)
schema_id = created_schema['schema_ids'][0]
print(schema_id)

## find schema_id

{'schema_ids': ['PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1']}
PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1


In [32]:
created_cred_def = response = await agent_controller.definitions.search_created(schema_issuer_did=my_did)
print(created_cred_def)
cred_def_id = created_cred_def['credential_definition_ids'][0]
print(cred_def_id)

## find cred_def_id

{'credential_definition_ids': ['PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default']}
PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default


In [33]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "author", "value": my_did},
    {"name": "origin", "value": my_did},
    {"name": "owner", "value": sp_did},
    {"name": "time", "value": time},
    {"name": "version", "value": "v1"},
    {"name": "name", "value": "balabababa music"},
    ]
record = await agent_controller.issuer.send_credential(music_connection, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)

Handle Credentials
Credential exchange a1239c73-2720-4255-bbef-a51c811866f7, role: issuer, state: offer_sent
Attributes: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': '8HWQruWXrv8s6BBLXm53B'}, {'name': 'time', 'value': '01-03-2023, 09:52:12'}, {'name': 'version', 'value': 'v1'}, {'name': 'name', 'value': 'balabababa music'}]
Handle Credentials
Credential exchange ab4f8dde-8bc8-4ebf-90ee-fa6cbdc17416, role: issuer, state: credential_acked
Attributes: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': '8HWQruWXrv8s6BBLXm53B'}, {'name': 'time', 'value': '01-03-2023, 09:50:36'}, {'name': 'version', 'value': 'v1'}, {'name': 'name', 'value': 'balabababa music'}]


In [34]:
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange a1239c73-2720-4255-bbef-a51c811866f7, role: issuer, state: offer_sent


### define new schema 

In [35]:
# Define you schema name - must be unique on the ledger
schema_name = "possession schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["possession", "time"]


In [36]:
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

PQRXDxdGqQGSZ8z69p4xZP:2:possession schema:0.0.1


In [37]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:14:default


### issue new VC to SP

In [38]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "possession", "value": music_did},
    {"name": "time", "value": time},
]
print(credential_attributes)

[{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 09:55:09'}]


In [39]:
record = await agent_controller.issuer.send_credential(sp_connection, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
print(record)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

{'credential_offer_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/offer-credential', '@id': '3a2fb28d-590b-4530-8928-c5291029bc51', '~thread': {}, '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'comment': 'create automated credential exchange', 'offers~attach': [{'@id': 'libindy-cred-offer-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJzY2hlbWFfaWQiOiAiUFFSWER4ZEdxUUdTWjh6NjlwNHhaUDoyOnBvc3Nlc3Npb24gc2NoZW1hOjAuMC4xIiwgImNyZWRfZGVmX2lkIjogIlBRUlhEeGRHcVFHU1o4ejY5cDR4WlA6MzpDTDoxNDpkZWZhdWx0IiwgImtleV9jb3JyZWN0bmVzc19wcm9vZiI6IHsiYyI6ICI1OTA1MjczNTMzODYxMTY2NjE5ODYxMjU5NzA5NzM5MTM5ODA3ODkxNTA3ODE1ODk3NDMwOTY3OTQ5ODQxNTU0MjY0NjEwOTUwODM2OSIsICJ4el9jYXAiOiAiMTA0NTI1MzcwODUzNjk2MzA0NDg0MTA2NjQzNDYyODUzODQ2NjgwODAyNDI4Mzc5OTQ0NjI1MzQ0NjM4NzYwODgxNTg4NjAwMTYxMTkwOTg3MTYyMzgxNTIyOTg5MDI3NzY0NDQyNDQwNzg4NTI5MTA0MjcyNjI2NzMyODkyNDg1OTY4MzcwOTk3NjcxNTE0MTMxNzIxMTY4MzY5OTUwMjU2NTYwMDI0ODU0OTE1NjA0MDAyMzUzNzYyMDY3MDYzMDg0NjE3OTg0Mjc0

In [76]:
response = await agent_controller.terminate()
print(response)

None
